# Preprocess and Load Data

In [ ]:
from ADUtils import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

## Load Data

In [ ]:
# Whether you already have data stored in a npy file
load_from_npy = False

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
# This loads data if it is not already stored in a npy file. 
if not load_from_npy:

    sequences, labels = [], []
    skip_sequence = False
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        if not os.path.exists(action_path):
            continue
        for sequence in np.array(os.listdir(action_path)).astype(int):
            window = []
            skip_sequence = False
            for frame_num in range(sequence_length):
                sequence_path = os.path.join(action_path, str(sequence))
                # 1. make sure path exists
                if not os.path.exists(sequence_path):
                    print("1")
                    skip_sequence = True
                    break
                # 2. make sure none are empty
                try:
                    res = np.load(os.path.join(sequence_path, "{}.npy".format(frame_num)))
                    window.append(res)
                except:
                    print(f"Cannot read {sequence_path} number {frame_num}")
                    print("2")
                    skip_sequence = True
                    break
            if not skip_sequence:
                # 3. make sure there is at least 30 frames
                if len(window) == 30:
                    sequences.append(window)
                    labels.append(label_map[action])
                    continue
                print("3")

## Description of Input Data

The tuple from the next command shows the dimension of our data. The details of the data is as follows. 

The first tuple displays the number of samples we collected. We collected samples for seven actions: 'raise_hand', 'thumbs_up', 'thumbs_down', 'cheer', 'cross_arms', 'clap', 'neutral'. For each of these actions, we captured variations of them to ensure we're covering variability for each action. This list describes the variation and number of samples for each:  
- raise_hand: 30 left hand only (15 hand on screen and 15 off), 30 right hand only (15 on screen and 15 off)
- thumbs_up: 30 left hand only, 30 right hand only, 30 both hands
- thumbs_down: 30 left hand only, 30 right hand only, 30 both hands 
- cheer: 30 hands below head, 30 hands above head 
- cross_arms: 30 no hands, 30 right hand only, 30 left hand only, 30 both hands
- clap: 30 right hand over left, 30 left hand over right, 30 both hands vertical to the screen
- neutral: 30 no hands 

The second element in the tuple shows that each sample has 30 frames. 

The last element in the tuple shows the number of data points per frame. These were extracted from the MediaPipe Holistic model. The features we are using include 33 pose landmarks (each of which have coordinates x, y, z, and visibility) and 21 hand landmarks (each have coordinates x, y, z). Moreover, we are collecing landmarks for both the left and the right hands. In total this is 258 coordinates (33 * 4 + 21 * 3 + 21 * 3). 

In [ ]:
print(np.array(sequences).shape)
# number of samples, number of frames per sample, number of data points per frame

## Splitting Data

In [ ]:
save_to_numpy = False
if save_to_numpy and not load_from_numpy:
    np.save('action_detect_X.npy', sequences)
    np.save('action_detect_y.npy', labels)

In [ ]:
if not load_from_npy:
    X = np.array(sequences)
else:
    X = np.load('action_detect.npy')

In [ ]:
if not load_from_npy:
    y = to_categorical(labels).astype(int)
else:
    y = np.load('action_detect_y')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

# Build and Train Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
# might remove

## Chosen Architecture

We used multiple LSTM layers stacked together to allow the model to learn higher-level temporal representations. Each layer potentially captures different aspects of temporal dependencies in the data. The ReLU activation function in the dense layers introduces non-linearity into the model, allowing it to learn more complex patterns in the data. The final dense layer uses a softmax activation for classification. We found that this amount of layers allowed the model to learn complex relationships in the data without overfitting. 

In [ ]:
model = Sequential()
# model.add(Input(shape=(30,258)))
model.add(Input(shape=X_train.shape[-2:]))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

In [ ]:
save_model = False
if save_model:
    model.save('action.keras')

## Validate Initial Model with 5-Fold-Cross-Validation

We used 5-fold-cross-validation to check if our model was overfitting. 

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
sequences = np.array(sequences)  
labels = np.array(labels)  

# Split the data
X_train_val, X_test, y_train_val, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # Validation is 20% of the original

if len(y_train.shape) == 1:  
    y_train = to_categorical(y_train, num_classes=actions.shape[0])
    y_val = to_categorical(y_val, num_classes=actions.shape[0])

In [ ]:
y_train_val = to_categorical(y_train_val, num_classes=np.unique(y_train_val).size)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
accuracies = []

for train_index, val_index in kf.split(X_train_val):
    # Split data
    X_train, X_val = X_train_val[train_index], X_train_val[val_index]
    y_train, y_val = y_train_val[train_index], y_train_val[val_index]
    # 
    model = Sequential()
    # model.add(Input(shape=(30,258)))
    model.add(Input(shape=X_train.shape[-2:]))
    model.add(LSTM(64, return_sequences=True, activation='relu'))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train model
    model.fit(X_train, y_train, epochs=100, verbose=0)
    
    # Evaluate model
    scores = model.evaluate(X_val, y_val, verbose=0)
    print(f"Score for fold {fold_no}: Accuracy of {scores[1]*100:.2f}%")
    accuracies.append(scores[1] * 100)
    fold_no += 1

average_score = np.mean(accuracies)
print(f"Average Score: {average_score:.2f}%")

## Training with Monitoring to Find Number of Epochs

This section lets you see which number of epochs may be best for specified batch size. 

As shown by 5-cross-fold-validation, our initial implementation was good. We wanted to see if it could be further improved with different number of epochs and batches. 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
# Callbacks for early stopping and saving the best model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
]

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks, verbose=1)

# Statistics

In [ ]:
# potentionally remove statistics generation

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, classification_report, f1_score

In [ ]:
y_prob = model.predict(X_test)

In [ ]:
# Uncomment this if 5-fold-cross-validation was used
# y_true = y_test.tolist() 
# y_pred = np.argmax(y_prob, axis=1).tolist()

In [ ]:
# Comment this out if 5-fold-cross-validation was used
y_true = np.argmax(y_test, axis=1).tolist()
y_pred = np.argmax(y_prob, axis=1).tolist()

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
classification_report(y_true, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt 
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=actions)
disp.plot(cmap='GnBu', xticks_rotation='vertical')
plt.show()

# Test in Real Time

In [ ]:
from tensorflow.keras.models import load_model
if save_model:
    model = load_model('action.keras')

In [ ]:
# change the colors
colors = [(245,117,16), (117,245,16), (16,117,245), (245,117,16), (117,245,16), (16,117,245), (245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
sequence = []
threshold = 0.5

# if you want to use video as testing
# video = '../data/test/thumbs_up/thumbs_up9.mp4'
# cap = cv2.VideoCapture(video)

# if you want a bigger video at the cost of performance
# cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        word = ''
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]
            print(actions[np.argmax(res)])
            
            word = actions[np.argmax(res)]
                
            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
        
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)

        cv2.putText(image, word, (3,30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()